In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as sm
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


Assignment 1:
    Catagorical variables: Product_Catagory & Quality
    Figure out reference category (class 3) (this is comparing smth to the others, so for example how much more expensive the food is compared to the other product categories)
    

In [2]:
df = pd.read_csv('1672182Pet shop.csv')
df.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size
0,1,162907,toys,premium,4.0,4,30.0,54.2,2744
1,2,195042,food,premium,3.9,4,22.0,64.4,3083
2,3,72749,toys,premium,3.2,9,29.0,41.5,1641
3,4,148853,other,off_brand,4.1,4,14.0,54.2,1832
4,5,192908,health,off_brand,4.2,3,5.0,31.2,2967


In [3]:
PS_dummies = pd.get_dummies(df[['product_category', 'quality']]) 
df = pd.concat([df, PS_dummies], axis = 1)
df.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,product_category_food,product_category_health,product_category_other,product_category_toys,quality_off_brand,quality_premium
0,1,162907,toys,premium,4.0,4,30.0,54.2,2744,False,False,False,True,False,True
1,2,195042,food,premium,3.9,4,22.0,64.4,3083,True,False,False,False,False,True
2,3,72749,toys,premium,3.2,9,29.0,41.5,1641,False,False,False,True,False,True
3,4,148853,other,off_brand,4.1,4,14.0,54.2,1832,False,False,True,False,True,False
4,5,192908,health,off_brand,4.2,3,5.0,31.2,2967,False,True,False,False,True,False


In [78]:
model1 = sm.ols('retail_price~product_category + products_sold + quality + satisfaction + discount + perc_physical + market_size + product_category_food + product_category_health + product_category_other + product_category_toys + quality_off_brand + quality_premium', data=df).fit() #reference category = currently food
print(model1.summary()) #is product_category the right choice for running this model?

                            OLS Regression Results                            
Dep. Variable:           retail_price   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     8.473
Date:                Sat, 30 Mar 2024   Prob (F-statistic):           1.64e-12
Time:                        15:57:00   Log-Likelihood:                -6510.7
No. Observations:                1941   AIC:                         1.304e+04
Df Residuals:                    1931   BIC:                         1.310e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

In [20]:
#df.dropna() alle non values weg doen
df.isnull().sum()

Unnamed: 0                   0
products_sold                0
product_category             0
quality                    109
satisfaction                 0
discount                     0
retail_price               155
perc_physical               67
market_size                  0
product_category_food        0
product_category_health      0
product_category_other       0
product_category_toys        0
quality_off_brand            0
quality_premium              0
dtype: int64

In [58]:
df_new = pd.get_dummies(df, dummy_na=True)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [59]:
df_imputed = IterativeImputer().fit_transform(df_new)
df_imputed = pd.DataFrame(df_imputed, columns=df_new.columns)
#replaced the NA's with estimates

#df_dropped = df.dropna()

In [79]:
model1_imputed = sm.ols('retail_price~products_sold + quality + satisfaction + discount + perc_physical + market_size + product_category_food + product_category_health + product_category_other + product_category_toys + quality_off_brand + quality_premium'', data=df_imputed).fit() #this wont work because df_new doesnt know a product_category table/row


SyntaxError: unterminated string literal (detected at line 1) (3180661057.py, line 1)

In [53]:
df_imputed.head()

,Unnamed: 0,products_sold,satisfaction,discount,retail_price,perc_physical,market_size,product_category_food,product_category_health,product_category_other,...,quality_off_brand,quality_premium,product_category_food,product_category_health,product_category_other,product_category_toys,product_category_nan,quality_off_brand,quality_premium,quality_nan
0,1.0,162907.0,4.0,4.0,30.0,54.2,2744.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2.0,195042.0,3.9,4.0,22.0,64.4,3083.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3.0,72749.0,3.2,9.0,29.0,41.5,1641.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,4.0,148853.0,4.1,4.0,14.0,54.2,1832.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,5.0,192908.0,4.2,3.0,5.0,31.2,2967.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [70]:
CooksD = model1_imputed.get_influence().cooks_distance #doesnt work yet!
n = len(df_imputed)
df_imputed['Outlier'] = CooksD[0] > 4/n
df_imputed.head()

,Unnamed: 0,products_sold,satisfaction,discount,retail_price,perc_physical,market_size,product_category_food,product_category_health,product_category_other,...,quality_premium,product_category_food,product_category_health,product_category_other,product_category_toys,product_category_nan,quality_off_brand,quality_premium,quality_nan,Outlier
0,1.0,162907.0,4.0,4.0,30.0,54.2,2744.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,False
1,2.0,195042.0,3.9,4.0,22.0,64.4,3083.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False
2,3.0,72749.0,3.2,9.0,29.0,41.5,1641.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,False
3,4.0,148853.0,4.1,4.0,14.0,54.2,1832.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,False
4,5.0,192908.0,4.2,3.0,5.0,31.2,2967.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,True


In [80]:
df_v2 = df[df_imputed.Outlier == False]


Assignment 2:

In [81]:
df_v2.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,product_category_food,product_category_health,product_category_other,product_category_toys,quality_off_brand,quality_premium
0,1,162907,toys,premium,4.0,4,30.0,54.2,2744,False,False,False,True,False,True
1,2,195042,food,premium,3.9,4,22.0,64.4,3083,True,False,False,False,False,True
2,3,72749,toys,premium,3.2,9,29.0,41.5,1641,False,False,False,True,False,True
3,4,148853,other,off_brand,4.1,4,14.0,54.2,1832,False,False,True,False,True,False
5,6,124273,health,premium,3.9,7,NaN,57.1,2525,False,True,False,False,False,True


In [85]:
df_v2.isnull().sum()

Unnamed: 0                   0
products_sold                0
product_category             0
quality                    102
satisfaction                 0
discount                     0
retail_price               155
perc_physical               60
market_size                  0
product_category_food        0
product_category_health      0
product_category_other       0
product_category_toys        0
quality_off_brand            0
quality_premium              0
dtype: int64

In [86]:
df_v2_new = pd.get_dummies(df_v2, dummy_na=True)

In [87]:
df_v2_new.head() #dealed with categorical missing data by adding a quality_nan list to show where data missing

,Unnamed: 0,products_sold,satisfaction,discount,retail_price,perc_physical,market_size,product_category_food,product_category_health,product_category_other,...,quality_off_brand,quality_premium,product_category_food,product_category_health,product_category_other,product_category_toys,product_category_nan,quality_off_brand,quality_premium,quality_nan
0,1,162907,4.0,4,30.0,54.2,2744,False,False,False,...,False,True,False,False,False,True,False,False,True,False
1,2,195042,3.9,4,22.0,64.4,3083,True,False,False,...,False,True,True,False,False,False,False,False,True,False
2,3,72749,3.2,9,29.0,41.5,1641,False,False,False,...,False,True,False,False,False,True,False,False,True,False
3,4,148853,4.1,4,14.0,54.2,1832,False,False,True,...,True,False,False,False,True,False,False,True,False,False
5,6,124273,3.9,7,NaN,57.1,2525,False,True,False,...,False,True,False,True,False,False,False,False,True,False


In [88]:
df_v2_imputed = IterativeImputer().fit_transform(df_v2_new)
df_v2_imputed = pd.DataFrame(df_v2_imputed, columns=df_v2_new.columns)

In [90]:
print(df_v2_imputed) #added imputed data for continuous missing data

#how to see where quality_nan = 1?????


      Unnamed: 0  products_sold  satisfaction  discount  retail_price  \
0            1.0       162907.0           4.0       4.0     30.000000   
1            2.0       195042.0           3.9       4.0     22.000000   
2            3.0        72749.0           3.2       9.0     29.000000   
3            4.0       148853.0           4.1       4.0     14.000000   
4            6.0       124273.0           3.9       7.0     21.197654   
...          ...            ...           ...       ...           ...   
2127      2246.0       112292.0           4.4       2.0     30.000000   
2128      2247.0       251999.0           4.7       4.0     16.000000   
2129      2248.0       108731.0           3.4       8.0     26.000000   
2130      2249.0       100125.0           4.1      10.0     34.000000   
2131      2250.0       121664.0           4.4       1.0     18.000000   

      perc_physical  market_size  product_category_food  \
0              54.2       2744.0                    0.0   
1    

! last step for assignment 2 is checking for multicollinearity 